Diabetes Prediction using Gradient Boosting Classifier

In [0]:
import pandas as pd
import numpy as np

In [2]:
# finding top features
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

data = pd.read_csv("diabetes.csv")
X = data.iloc[:,0:8]  #independent columns
y = data.iloc[:,-1]

#apply SelectKBest class to extract top 10 best features
bestfeatures = SelectKBest(score_func=chi2, k=5)
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores.nlargest(10,'Score'))  #print 10 best features

                      Specs        Score
4                   Insulin  2175.565273
1                   Glucose  1411.887041
7                       Age   181.303689
5                       BMI   127.669343
0               Pregnancies   111.519691
3             SkinThickness    53.108040
2             BloodPressure    17.605373
6  DiabetesPedigreeFunction     5.392682


In [0]:
df = pd.read_csv("diabetes.csv")
df.rename(columns={"DiabetesPedigreeFunction": "Dpf"}, inplace=True)
# i will be using only top 5 features so i'm gonna drop the rest
df.drop(['BloodPressure', 'SkinThickness', 'Dpf'], 1, inplace=True)
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,Dpf,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [0]:
df.isnull().values.any()

False

In [0]:
# filling zero values with average scores of each columns, also depending on the outcome

def fill_zeros(df, field, target):
    mean_by_target = df.loc[df[field] != 0, [field, target]].groupby(target).mean()
    df.loc[(df[field] == 0)&(df[target] == 0), field] = mean_by_target.iloc[0][0]
    df.loc[(df[field] == 0)&(df[target] == 1), field] = mean_by_target.iloc[1][0]

for col in ['Glucose', 'Insulin', 'BMI']:   
    fill_zeros(df, col, 'Outcome') 

In [0]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
x = df[['Pregnancies', 'Glucose', 'Insulin', 'BMI', 'Age']]
y = df['Outcome']
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)
classifier = GradientBoostingClassifier(random_state=0)
classifier.fit(X_train, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [0]:
from sklearn.metrics import accuracy_score
# make predictions for test data
y_pred = classifier.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 89.61%


In [0]:
import pickle
with open('diabetes.pkl', 'wb') as file:
    pickle.dump(classifier, file)